# MNL probability work

Sam Maurer, Sep 2018

This notebook is for feature development related to ChoiceModels [issue #26](https://github.com/UDST/choicemodels/issues/26)

### Set up an MNL model using fake data

In [1]:
import numpy as np
import pandas as pd

import orca

from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep

In [2]:
d1 = {'oid': np.arange(100), 
      'obsval': np.random.random(100),
      'choice': np.random.choice(np.arange(3), size=100)}

d2 = {'aid': np.arange(3), 
      'altval': np.random.random(3)}

obs = pd.DataFrame(d1).set_index('oid')
orca.add_table('obs', obs)

alts = pd.DataFrame(d2).set_index('aid')
orca.add_table('alts', alts)

In [3]:
mm.initialize()

No yaml files found in path 'configs'


In [4]:
m = LargeMultinomialLogitStep()
m.choosers = 'obs'
m.alternatives = 'alts'
m.choice_column = 'choice'
m.model_expression = 'obsval + altval - 1'

m.fit()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:            100
Model:         Multinomial Logit   Df Residuals:                 98
Method:       Maximum Likelihood   Df Model:                      2
Date:                 2018-09-11   Pseudo R-squ.:             0.002
Time:                      14:42   Pseudo R-bar-squ.:        -0.027
AIC:                     142.388   Log-Likelihood:          -69.194
BIC:                     147.598   LL-Null:                 -69.315
             coef   std err         z     P>|z|   Conf. Int.
------------------------------------------------------------
obsval     0.0000     0.354     0.000     1.000             
altval    -0.2342     0.328    -0.713     0.476             


/Users/maurer/Dropbox/Git-mbp13/udst/choicemodels/choicemodels/mnl.py:236: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  chosen = np.reshape(self._df[[self._choice_col]].as_matrix(),
/Users/maurer/Dropbox/Git-mbp13/udst/choicemodels/choicemodels/mnl.py:239: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  log_lik, fit = mnl_estimate(model_design.as_matrix(), chosen, self._numalts)


In [5]:
m.fitted_parameters

[1.3935816521953695e-16, -0.23421124250743391]

### Refactor MNL probability calculations

These are moving from a set of functions in `urbansim.urbanchoice.mnl` to a method of the `choicemodels.MultinomialLogitResults` class.

In [6]:
import choicemodels
from urbansim.urbanchoice import mnl

import patsy

In [7]:
mct = choicemodels.tools.MergedChoiceTable(obs[:2], alts, sample_size=5)
print(len(mct.to_frame()))
print(mct.to_frame().head())

10
           obsval  choice    altval
oid aid                            
0   0    0.323629       2  0.718287
    2    0.323629       2  0.748681
    1    0.323629       2  0.079932
    1    0.323629       2  0.079932
    1    0.323629       2  0.079932


In [8]:
# Get probabilities using urbansim.urbanchoice.mnl

dm = patsy.dmatrix(m.model_expression, data=mct.to_frame(),
                   return_type='dataframe')

probs = mnl.mnl_simulate(data=dm, coeff=m.fitted_parameters,
                         numalts=mct.sample_size, returnprobs=True)

df = mct.to_frame()
df['prob'] = np.reshape(probs, (probs.size, 1))

In [9]:
df.head()

obsval  choice    altval      prob
oid aid                                      
0   0    0.323629       2  0.718287  0.182592
    2    0.323629       2  0.748681  0.181296
    1    0.323629       2  0.079932  0.212037
    1    0.323629       2  0.079932  0.212037
    1    0.323629       2  0.079932  0.212037

In [10]:
# Refactored codebase

results = choicemodels.MultinomialLogitResults('ChoiceModels', 
                m.model_expression, fitted_parameters=m.fitted_parameters)

probs = results.probabilities(mct)

In [11]:
probs

oid  aid
0    0      0.182592
     2      0.181296
     1      0.212037
     1      0.212037
     1      0.212037
1    1      0.218470
     2      0.186797
     0      0.188131
     1      0.218470
     0      0.188131
Name: prob, dtype: float64

### Check performance

In [15]:
d1 = {'oid': np.arange(1000000), 
      'obsval': np.random.random(1000000)}

test_obs = pd.DataFrame(d1).set_index('oid')

In [19]:
mct = choicemodels.tools.MergedChoiceTable(test_obs, alts, sample_size=10)
print(len(mct.to_frame()))
print(mct.to_frame().head())

10000000
           obsval    altval
oid aid                    
0   2    0.656677  0.748681
    1    0.656677  0.079932
    0    0.656677  0.718287
    2    0.656677  0.748681
    1    0.656677  0.079932


In [20]:
%%time

dm = patsy.dmatrix(m.model_expression, data=mct.to_frame(),
                   return_type='dataframe')

probs = mnl.mnl_simulate(data=dm, coeff=m.fitted_parameters,
                         numalts=mct.sample_size, returnprobs=True)

df = mct.to_frame()
df['prob'] = np.reshape(probs, (probs.size, 1))

CPU times: user 907 ms, sys: 412 ms, total: 1.32 s
Wall time: 1.2 s


In [21]:
%%time

probs = results.probabilities(mct)

CPU times: user 1.01 s, sys: 432 ms, total: 1.44 s
Wall time: 1.37 s
